<a href="https://colab.research.google.com/github/VardanDavtyan/ML-DL/blob/main/Fake_News_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import random

from nltk.stem.porter import PorterStemmer

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
fake = pd.read_csv('Fake.csv')
fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [ ]:
fake["subject"].value_counts()

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

In [ ]:
real = pd.read_csv('True.csv')
real

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
unknown_publishers = []
for index, row in enumerate(real.text.values):
  try:
    record = row.split('-', maxsplit=1)
    record[1]
    assert(len(record[0])<120)
  except:
    unknown_publishers.append(index)



In [ ]:
real.iloc[unknown_publishers].text

7        The following statements were posted to the ve...
8        The following statements were posted to the ve...
12       The following statements were posted to the ve...
13       The following statements were posted to the ve...
14        (In Dec. 25 story, in second paragraph, corre...
                               ...                        
20135     (Story corrects to million from billion in pa...
20500     (This Sept 8 story corrects headline, clarifi...
20667     (Story refiles to add dropped word  not , in ...
21246     (Story corrects third paragraph to show Mosul...
21339     (Story corrects to fix spelling in paragraph ...
Name: text, Length: 222, dtype: object

In [ ]:
publisher = []
tmp_text = []

for index, row in enumerate(real.text.values):
  if index in unknown_publishers:
    tmp_text.append(row)
    publisher.append('Unknown')
  else:
    record = row.split('-', maxsplit=1)
    publisher.append(record[0].strip())
    tmp_text.append(record[1].strip())


In [ ]:
real['publisher'] = publisher
real['text'] = tmp_text

In [ ]:
real.head()

,title,text,subject,date,publisher
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,politicsNews,"December 31, 2017",WASHINGTON (Reuters)
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,politicsNews,"December 29, 2017",WASHINGTON (Reuters)
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,politicsNews,"December 31, 2017",WASHINGTON (Reuters)
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,politicsNews,"December 30, 2017",WASHINGTON (Reuters)
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...,politicsNews,"December 29, 2017",SEATTLE/WASHINGTON (Reuters)


In [ ]:
empty_fake_index = [index for index, text in enumerate(fake.text.tolist()) if str(text).strip() == ""]

In [ ]:
fake.iloc[empty_fake_index]

,title,text,subject,date
10923,TAKE OUR POLL: Who Do You Think President Trum...,,politics,"May 10, 2017"
11041,Joe Scarborough BERATES Mika Brzezinski Over “...,,politics,"Apr 26, 2017"
11190,WATCH TUCKER CARLSON Scorch Sanctuary City May...,,politics,"Apr 6, 2017"
11225,MAYOR OF SANCTUARY CITY: Trump Trying To Make ...,,politics,"Apr 2, 2017"
11236,SHOCKER: Public School Turns Computer Lab Into...,,politics,"Apr 1, 2017"
...,...,...,...,...
21816,BALTIMORE BURNS: MARYLAND GOVERNOR BRINGS IN N...,,left-news,"Apr 27, 2015"
21826,FULL VIDEO: THE BLOCKBUSTER INVESTIGATION INTO...,,left-news,"Apr 25, 2015"
21827,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,left-news,"Apr 25, 2015"
21857,(VIDEO)ICE PROTECTING OBAMA: WON’T RELEASE NAM...,,left-news,"Apr 14, 2015"


In [ ]:
real['text'] = real['title'] + " " + real['text']
fake['text'] = fake['title'] + " " + fake['text']

real['class'] = 1
fake['class'] = 0

real = real[['text', 'class']]
fake = fake[['text', 'class']]

In [ ]:
data = real.append(fake, ignore_index=True)
data

,text,class
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1
...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0
44896,How to Blow $700 Million: Al Jazeera America F...,0


In [ ]:
ps = PorterStemmer()

def preprocess_text(_text):
  _text = _text.lower()
  _text = re.sub(r'\$[^\s]+', 'dollar', _text)
  _text = re.sub(r'[^a-z0-9\s]', '', _text)
  _text = re.sub(r'[0-9]+', 'number', _text)
  _text = _text.split(" ")
  _text = list(map(lambda x: ps.stem(x), _text))
  _text = list(map(lambda x: x.strip(), _text))
  if '' in _text:
    _text.remove('')

  return _text

data.text = data['text'].apply(preprocess_text)
data

,text,class
0,"[as, us, budget, fight, loom, republican, flip...",1
1,"[us, militari, to, accept, transgend, recruit,...",1
2,"[senior, us, republican, senat, let, mr, muell...",1
3,"[fbi, russia, probe, help, by, australian, dip...",1
4,"[trump, want, postal, servic, to, charg, much,...",1
...,...,...
44893,"[mcpain, john, mccain, furiou, that, iran, tre...",0
44894,"[justic, yahoo, settl, email, privaci, classac...",0
44895,"[sunnistan, us, and, alli, safe, zone, plan, t...",0
44896,"[how, to, blow, dollar, million, al, jazeera, ...",0


In [ ]:
import gensim

y = data['class'].values
x = [d for d in data['text'].tolist()]
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=x, size=DIM, window=10, min_count=1)


In [ ]:
w2v_model.wv.most_similar('book')

[('memoir', 0.6993789672851562),
 ('tellal', 0.6469038128852844),
 ('essay', 0.6402901411056519),
 ('novel', 0.62357497215271),
 ('titl', 0.6078211665153503),
 ('poem', 0.6048157811164856),
 ('biographi', 0.5996989011764526),
 ('column', 0.5973190665245056),
 ('articl', 0.5920922756195068),
 ('magazin', 0.5643718242645264)]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)

In [ ]:
nos = np.array([len(i) for i in x])
maxlen = 1000
x = pad_sequences(x, maxlen=maxlen)
vocab_size = len(tokenizer.word_index) + 1
vocab = tokenizer.word_index

In [ ]:
def get_weight_matrix(model):

  weight_matrix = np.zeros((vocab_size, DIM))

  for word, i in vocab.items():
    weight_matrix[i] = model.wv[word]

  return weight_matrix

In [ ]:
embedding_vectors = get_weight_matrix(w2v_model)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         19052500  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 19,169,877
Trainable params: 117,377
Non-trainable params: 19,052,500
_________________________________________________________________


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
model.fit(x_train, y_train, validation_split=0.3, epochs=12)

Epoch 1/12
737/737 [==============================] - 48s 57ms/step - loss: 0.1104 - acc: 0.9592 - val_loss: 0.0618 - val_acc: 0.9798
Epoch 2/12
737/737 [==============================] - 40s 54ms/step - loss: 0.0432 - acc: 0.9856 - val_loss: 0.0265 - val_acc: 0.9909
Epoch 3/12
737/737 [==============================] - 40s 55ms/step - loss: 0.0184 - acc: 0.9943 - val_loss: 0.0238 - val_acc: 0.9912
Epoch 4/12
737/737 [==============================] - 43s 58ms/step - loss: 0.0117 - acc: 0.9964 - val_loss: 0.0157 - val_acc: 0.9950
Epoch 5/12
737/737 [==============================] - 43s 59ms/step - loss: 0.0125 - acc: 0.9960 - val_loss: 0.0289 - val_acc: 0.9900
Epoch 6/12
737/737 [==============================] - 41s 56ms/step - loss: 0.0132 - acc: 0.9949 - val_loss: 0.0139 - val_acc: 0.9955
Epoch 7/12
737/737 [==============================] - 42s 57ms/step - loss: 0.0030 - acc: 0.9994 - val_loss: 0.0120 - val_acc: 0.9963
Epoch 8/12
737/737 [==============================] - 41s 55ms

In [ ]:
y_pred = (model.predict(x_test) >= 0.5).astype(int)
y_pred

351/351 [==============================] - 7s 20ms/step


array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
accuracy_score(y_test, y_pred) * 100

99.46547884187082

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5824
           1       0.99      1.00      0.99      5401

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



## Testing

In [ ]:

news = input("Enter some news: ")
some_text = [news]
some_text = tokenizer.texts_to_sequences(some_text)
some_text = pad_sequences(some_text, maxlen=maxlen)

print("News are " + ("True" if (model.predict(some_text) >= 0.5).astype(int) == 1 else "Fake"))

Enter some news: Trump Suggests Americans Can Inject Themselves with Disinfectant to Cure Coronavirus
1/1 [==============================] - 0s 43ms/step
News are Fake
